### Imports

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


2024-08-01 23:24:18.253742: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 23:24:18.255488: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 23:24:18.260722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 23:24:18.269418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 23:24:18.271945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 23:24:18.278811: I tensorflow/core/platform/cpu_feature_gu

### Data
Load data and produce train/test split 

In [2]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)




In [3]:
print(data)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.0              0.27         0.36            20.7      0.045   
1               6.3              0.30         0.34             1.6      0.049   
2               8.1              0.28         0.40             6.9      0.050   
3               7.2              0.23         0.32             8.5      0.058   
4               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4893            6.2              0.21         0.29             1.6      0.039   
4894            6.6              0.32         0.36             8.0      0.047   
4895            6.5              0.24         0.19             1.2      0.041   
4896            5.5              0.29         0.30             1.1      0.022   
4897            6.0              0.21         0.38             0.8      0.020   

      free sulfur dioxide  

### Define model
Define model architecture and train model. MLFlow will track model parameters, results and artefacts.

In [4]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [5]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


### Hyperparameter Search

In [6]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


### Run Model on MLFlow

In [7]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

  0%|          | 0/8 [00:00<?, ?trial/s, best loss=?]

I0000 00:00:1722551060.479730   27726 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-01 23:24:20.479942: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


 1/46 ━━━━━━━━━━━━━━━━━━━━ 17s 379ms/step - loss: 38.2389 - root_mean_squared_error: 6.1838
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 36.9093 - root_mean_squared_error: 6.0753 - val_loss: 36.6372 - val_root_mean_squared_error: 6.0529

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 38.4569 - root_mean_squared_error: 6.2014
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 36.6310 - root_mean_squared_error: 6.0523 - val_loss: 36.0606 - val_root_mean_squared_error: 6.0050

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 34.2408 - root_mean_squared_error: 5.8516
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35.4146 - root_mean_squared_error: 5.9510 - val_loss: 35.4942 - val_root_mean_squared_error: 5.9577

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 35.9275 - root_mean_squared_error: 5.9940
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35.3683 - root_mean_squared_error: 5.

/home/jared/.local/share/virtualenvs/ml-xp-lovsBfPe/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(



 1/46 ━━━━━━━━━━━━━━━━━━━━ 11s 253ms/step - loss: 37.0900 - root_mean_squared_error: 6.0902
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.0624 - root_mean_squared_error: 6.0051 - val_loss: 34.2339 - val_root_mean_squared_error: 5.8510

Epoch 2/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 34.8864 - root_mean_squared_error: 5.9065
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 33.4269 - root_mean_squared_error: 5.7813 - val_loss: 31.4559 - val_root_mean_squared_error: 5.6086

Epoch 3/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 32.5302 - root_mean_squared_error: 5.7035
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30.6962 - root_mean_squared_error: 5.5403 - val_loss: 28.9092 - val_root_mean_squared_error: 5.3767

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 29.7702 - root_mean_squared_error: 5.4562
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 976u

In [8]:
!mlflow models serve -m "models:/wine-quality/1" --port 5002


2024/08/01 23:38:54 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/08/01 23:38:54 INFO mlflow.utils.virtualenv: Installing python 3.10.4 if it does not exist
2024/08/01 23:38:54 INFO mlflow.utils.virtualenv: Creating a new environment in /home/jared/.mlflow/envs/mlflow-c1dc92e94d5f13c9c48812cf62c26ea0d5351905 with /home/jared/.pyenv/versions/3.10.4/bin/python
/home/jared/.local/share/virtualenvs/ml-xp-lovsBfPe/bin/python: No module named virtualenv
2024/08/01 23:38:54 WARNING mlflow.utils.virtualenv: Encountered an unexpected error: ShellCommandException("Non-zero exit code: 1\nCommand: ['/home/jared/.local/share/virtualenvs/ml-xp-lovsBfPe/bin/python', '-m', 'virtualenv', '--python', '/home/jared/.pyenv/versions/3.10.4/bin/python', '/home/jared/.mlflow/envs/mlflow-c1dc92e94d5f13c9c48812cf62c26ea0d5351905']") while creating a virtualenv environment in /home/jared/.mlflow/envs/mlflow-c1dc92e94d5f13c9c48812cf62c26ea0d5351905, removing the en